In [159]:
import numpy as np
from sklearn import datasets
from sklearn import preprocessing

In [160]:
# User-driven regression nn with one hidden layer, leaky-ReLu activation, MSE
# on sklearn's diabetes dataset
# adding a 2nd hidden layer

num_hid_units = 8
num_hid_units2 = 8
num_features = 2
a = 0.000005

# house_sk_data = datasets.load_boston()
# # size of training and cross validation sets
# m_train = 400
# m_cv = 506 - m_train

# either dataset works
house_sk_data = datasets.load_diabetes()
m_train = 266
m_cv = 133
m_test = 43

# training set
house_sk_train = preprocessing.scale(house_sk_data.data[:m_train, :num_features])
house_sk_train_bias = np.column_stack([np.ones((m_train, 1)), house_sk_train])

# CV set
house_sk_cv = preprocessing.scale(house_sk_data.data[m_train:(m_train+m_cv), :num_features])
house_sk_cv_bias = np.column_stack([np.ones((m_cv, 1)), house_sk_cv])

# test set
house_sk_test = preprocessing.scale(house_sk_data.data[(m_train+m_cv):(m_train+m_cv+m_test), :num_features])
house_sk_test_bias = np.column_stack([np.ones((m_test, 1)), house_sk_test])

# target set
house_sk_target = house_sk_data.target[:m_train]
house_sk_cv_target = house_sk_data.target[m_train:(m_train+m_cv)]
house_sk_test_target = house_sk_data.target[(m_train+m_cv):(m_train+m_cv+m_test)]

In [161]:
# randomly initialize weights for first, second, and third layers
weights = np.random.rand(num_hid_units, num_features+1)
weights2 = np.random.rand(num_hid_units2, num_hid_units+1)
weights3 = np.random.rand(1, num_hid_units2+1)


# Forward and backward propagates for a given training example _i_
def propagate(i):
    global weights3, weights2, weights

    # learning rate
    # 0.0003 retains lowest MSE for CV set

    # z for second layer and activation (leaky-ReLu)
    z2 = np.dot(weights, np.transpose(house_sk_train_bias[i, :]))
    a2 = np.where(z2 > 0, z2, 0.01 * z2)
    # add bias unit for second layer
    a2_bias = np.concatenate((np.ones(1), a2))
    # z for third layer and final activation (leaky-ReLu)
    z3 = np.dot(weights2, a2_bias)
    a3 = np.where(z3 > 0, z3, 0.01 * z3)
    # add bias unit for third layer
    a3_bias = np.concatenate((np.ones(1), a3))
    # linear hypothesis
    hypot = np.dot(weights3, a3_bias)
    print('hypot: ', hypot)
    # Backpropagates to accumulate partials and deltas
    # Delta for fourth layer (mean squared error)
    delta4 = hypot - house_sk_target[i]
    # derivative of error w.r.t weights, third layer
    layer3partials = np.multiply(delta4, a3_bias)

    # derivative of error w.r.t weights, second layer
    layer3partials_nobias = layer3partials[1:]
    layer2partials = np.dot(layer3partials_nobias.reshape((-1,1)), np.transpose(a2_bias.reshape((-1,1))))

    # derivative of error w.r.t weights, first layer
    layer2partials_nobias = layer2partials[:, 1:]
    #feat_bias_matrix = np.row_stack([house_sk_train_bias[i, :] for n in range(num_hid_units)])
    feat_bias_matrix = house_sk_train_bias[i, :]
    print('layer 2 partials: ', np.shape(layer2partials_nobias))
    print('feat bias: ', np.shape(feat_bias_matrix))
    print('weights1: ', np.shape(weights))
    layer1partials = np.dot(layer2partials_nobias, feat_bias_matrix)

    weights3 = weights3 - (a * layer3partials)
    weights2 = weights2 - (a * layer2partials)
    weights = weights - (a * layer1partials)

In [162]:
# Train weights
for z in range(0, m_train):
    propagate(z)

hypot:  [33.06044735]
layer 2 partials:  (8, 8)
feat bias:  (3,)
weights1:  (8, 3)


ValueError: shapes (8,8) and (3,) not aligned: 8 (dim 1) != 3 (dim 0)

In [ ]:
# predict example in given set
def predict(j, pred_set, target_set):
    # z for second layer and activation (ReLu)
    z2 = np.dot(weights, np.transpose(pred_set[j, :]))
    # print('z2', z2)
    a2 = np.where(z2 > 0, z2, 0.01 * z2)
    # print('a2', a2)
    # add bias unit for second layer
    a2_bias = np.concatenate((np.ones(1), a2))
    # z for third layer, no final activation for now
    z3 = np.dot(weights3, a2_bias)

    # calculate error
    sq_error = (np.square(z3 - target_set[j]))
    print(f'Prediction: {z3}, actual: {target_set[j]}, error: {0.5 * sq_error}')
    return sq_error

In [ ]:
tot_err_train = 0
# Predict in training example and see average error
for y in range(0, m_train):
    tot_err_train += predict(y, house_sk_train_bias, house_sk_target)

print('=========CV-SET=========')
tot_err = 0
# Predict CV example and see average error
for x in range(0, m_cv):
    tot_err += predict(x, house_sk_cv_bias, house_sk_cv_target)

print('MSE training ($): ', tot_err_train / (2 * m_train))
print('MSE CV ($): ', tot_err / (2 * m_cv))
